In [2]:
import os
import json


In [3]:
# Open and read the config file
with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

# Retrieve the API key from the config data
api_key = config_data['api_key']
os.environ['OPENAI_API_KEY'] = api_key

## Load the index

In [4]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_storage(storage_context)

## Query index

In [4]:
# create query engine
# it will use lazy embedding
queryEngine = index.as_query_engine()

In [5]:
queryEngine.query("how do I plan a tunneling project in LiquidEarth? Try finding the Control Fields for each steps of the workflow and append them as a list in order of the operations to your response. only include controls that you are confident exist and are relevant to the question. Only answer this question if you can find the answer in the documentation. If no documentation is found, ask the user to rephrase the question.")

Response(response='\n1. Select the “Tunneling” option from the “Projects” tab.\n2. Select the “Create” button to start a new project.\n3. Select the “Import” button to upload the 3D geological data.\n4. Select the “Analyze” button to analyze the data.\n5. Select the “Design” button to design the tunneling project.\n6. Select the “Simulate” button to simulate the project.\n7. Select the “Export” button to export the project.', source_nodes=[NodeWithScore(node=TextNode(id_='f15bb996-8bcd-4113-b0fb-e3e87b27debd', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2aebf52a-532c-4862-983f-830390801537', node_type=None, metadata={}, hash='e23fdfe472529d28782acc176c21be8a7053496336abdf00326a04e7c106a788')}, hash='97bbeea97343bad39518559157513c9a3250c3b321b872880aba521b8b7e14af', text='LiquidEarth Documentation\n\nAll things related to the product - processes, best practices, setup

## Query with a prompt template

In [5]:
from llama_index import Prompt
# define custom Prompt
TEMPLATE_STR = (
    "We have provided Documentation on the software and further information below. The Documentation includes Control properties, with the name of the UI element for a certain functionality within the LiquidEarth App .\n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Answer the question for a human to understand. Additionally, ONLY where controls are available and, return the control properties in the order of operations in the following form at the end of your response: [[control1], [control2], ...]. Never include any controls that are empty or not specifically included as Control properties in the provided documentation. Given this information, please answer the question: {query_str}\n"
)
QA_TEMPLATE = Prompt(TEMPLATE_STR)

# Configure query engine
query_engine = index.as_query_engine(text_qa_template=QA_TEMPLATE)

# Execute query
response = query_engine.query("how can I create a project?")
print(response)


To create a project, open up the server explorer on the left and click the Plus button to create a new project. It will appear at the bottom of your server explorer list. To edit the project's basic information and metadata, use the Edit Data control in the inspector. Once you have set all required fields correctly, save them to accept the information. Then, use the Load Project control in the inspector to load the project into your local workspace (3D Workspace). It will appear in your 3D workspace and listed in the Local Explorer. [[Plus], [Edit Data], [Save], [Load Project]].


In [28]:
response = query_engine.query("how to install liquidEarth on my quest?")
print(response)

## Chat with the index

In [9]:
chat_engine = index.as_chat_engine()
response = chat_engine.chat("what is LiquidEarth?")
print(response)


LiquidEarth is a cloud-based solution for the visualization, processing, and analysis of 3D geological data, models, and more. It is relevant to the conversation we were having because it provides a way to visualize, process, and analyze 3D geological data, models, and more.


In [10]:
response = chat_engine.chat("what was my previous question?")
print(response)


LiquidEarth is a cloud-based solution for the visualization, processing, and analysis of 3D geological data, models, and more. It is relevant to the conversation because it provides a way to visualize, process, and analyze 3D geological data, models, and more.


In [11]:
response = chat_engine.chat("can you rephrase my previous question?")

In [12]:
print(response)


LiquidEarth is a cloud-based solution for the visualization, processing, and analysis of 3D geological data, models, and more. It is relevant to the conversation because it provides a way to visualize, process, and analyze 3D geological data, models, and more.


In [13]:
response = chat_engine.chat("please repeat my previous question backwards")
print(response)


What can I do if I need help with LiquidEarth, the cloud-based solution for the visualization, processing, and analysis of 3D geological data, models, and more?


In [14]:
chat_engine.reset()

## Chat with a prompt template (ToDo)

In [ ]:
custom_prompt = Prompt("""\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
""")

# list of (human_message, ai_message) tuples
custom_chat_history = [
    (
        'Hello assistant, we are having a insightful discussion about Paul Graham today.',
        'Okay, sounds good.'
    )
]

query_engine = index.as_query_engine()
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    verbose=True
)

## GPT 4

In [6]:
from llama_index.llms import OpenAI
from llama_index import ServiceContext

gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

In [7]:
query_engine = index.as_query_engine(service_context=service_context_gpt4, text_qa_template=QA_TEMPLATE)

In [8]:
response = query_engine.query("how can I create a project?")

In [9]:
print(response)


To create a new project in the LiquidEarth App, follow these steps:

1. Open the server explorer which is located on the left side of the interface.
2. Click on the Plus button. This will create a new project which will appear at the bottom of your server explorer list.
3. Optionally, you can edit the basic information and project metadata such as the project name and owner. To do this, use the Edit Data option in the inspector. Once you have set all required fields correctly, click Save to accept the information.
4. To load the project into your local workspace (3D Workspace), use the Load Project option in the inspector. The project will appear in your 3D workspace and will be listed in the Local Explorer. If you are working online, the project will also be automatically uploaded to the cloud or the remote server you are connected to.

The control properties in the order of operations are: [[Server Explorer], [Plus Button], [Edit Data], [Save], [Load Project]].


In [11]:
response = query_engine.query("i want to visualize my leapfrog geological model. how can I do that?")
print(response)

To visualize your Leapfrog geological model in the LiquidEarth app, you would first need to import your model into the app. Unfortunately, the provided documentation does not specify the exact steps or controls to import and visualize a Leapfrog geological model. 

However, once your model is imported, you can use the Real-Scale View feature to visualize your model in real scale. This feature allows you to place your view in the model or on top of a rendered topography. If you're using AR, it can also use your GPS to create a localized real-world overlay of your model.

If you're new to the app, it's recommended to check out the “Getting Started” section in the user manual. If you need further assistance, you can contact support at support@terranigma-solutions.com.

Since the documentation does not provide specific control properties for this process, I'm unable to provide them in the requested format.
